# LSTM Sequence Classification
Reference: https://towardsdatascience.com/word-bags-vs-word-sequences-for-text-classification-e0222c21d2ec

Import Libraries:

In [36]:
import numpy as np
import os
import time
from datetime import datetime
import pandas as pd
import json
import re
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import StratifiedShuffleSplit
import random as rn
import keras
import tensorflow as tf
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

startTime = time.time()
print("Tensor Flow Version: ",tf.__version__)

Tensor Flow Version:  2.2.0-rc2


In [37]:
#All this for reproducibility
np.random.seed(1)
rn.seed(1)
tf.random.set_seed(1)
tf.config.threading.set_intra_op_parallelism_threads(1)
tf.config.threading.set_inter_op_parallelism_threads(112)

In [38]:
# Build the corpus and sequences
labelToName = { 0 : 'Rejected', 1 : 'Allowed' }
namesInLabelOrder = ['Rejected', 'Allowed']

#Read
readFromFile = np.loadtxt('EncodedCriminalBailApplication.csv', delimiter=',')
# print the array
print("\nTest Read :",readFromFile[1])

Xencoded=readFromFile[:,:-1]
print("\nXencoded :",Xencoded[1])
labels=readFromFile[:,-1]
print("\nlabels :",labels[1])


Test Read : [   0.    0.    0. ...  358. 1575.    0.]

Xencoded : [   0.    0.    0. ...   18.  358. 1575.]

labels : 0.0


In [39]:
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=1).split(Xencoded, labels)
train_indices, test_indices = next(sss)
print(np.shape(train_indices))
print(np.shape(test_indices))

train_x = Xencoded[train_indices]
test_x = Xencoded[test_indices]
train_labels = keras.utils.to_categorical(labels[train_indices], len(labelToName))
test_labels = keras.utils.to_categorical(labels[test_indices], len(labelToName))

(309,)
(78,)


In [40]:
# Build the LSTM model
def getModel():
    units1, units2 = int (nWords/4), int (nWords/8)
    model = keras.models.Sequential()
    model.add(keras.layers.embeddings.Embedding(input_dim = nWords+1,output_dim=units1,input_length=maxsequenceLength, trainable=True))               
    model.add(keras.layers.LSTM(units = units2, return_sequences =False))                              
    model.add(keras.layers.Dense(len(labelToName), activation ='softmax'))                           
    model.compile(optimizer='adam', loss = 'categorical_crossentropy', metrics=['acc'])
    return model

In [41]:
# Train and test over multiple train/validation sets
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=2, mode='auto', restore_best_weights=False)                         
sss2 = StratifiedShuffleSplit(n_splits=10, test_size=0.2, random_state=1).split(train_x, train_labels)          
results={}
for i in range(2):
    train_indices_2, val_indices = next(sss2)
    model = getModel()
    model.summary()
    history=model.fit(x=train_x[train_indices_2], y=train_labels[train_indices_2], epochs=50, batch_size=32, shuffle=True, validation_data = (train_x[val_indices], train_labels[val_indices]), verbose=2, callbacks=[early_stop], use_multiprocessing=True)
    test_loss, test_accuracy = model.evaluate(test_x, test_labels, verbose=2)                                
    print ("\ntest_loss:",test_loss, "\ntest_accuracy:",test_accuracy)
    predicted = model.predict(test_x, verbose=2)
    predicted_labels = predicted.argmax(axis=1)        
    print ("\n\nConfusion Matrix:\n",confusion_matrix(labels[test_indices], predicted_labels))
    print ("\n\nClassification Report:\n\n",classification_report(labels[test_indices], predicted_labels, digits=4, target_names=namesInLabelOrder))
    result={}
    result['confusion_matrix'] = confusion_matrix(labels[test_indices], predicted_labels).tolist()
    result['classification_report'] = classification_report(labels[test_indices], predicted_labels, digits=4, target_names=namesInLabelOrder, output_dict=True)
    results[i] = result
os.chdir('D:\\PhD\\Dataset\\Results')
filename = 'LSTM-'+time.strftime("%d-%m-%Y-%H-%M-%S")+'.json'
f = open (filename,'w')
out = json.dumps(results, ensure_ascii=True)
f.write(out)
f.close()

endTime = time.time()
hours, rem = divmod(endTime-startTime, 3600)
minutes, seconds = divmod(rem, 60)
print("\n\nTotal time taken: ")
print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 1503, 2550)        26020200  
_________________________________________________________________
lstm_3 (LSTM)                (None, 1275)              19512600  
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 2552      
Total params: 45,535,352
Trainable params: 45,535,352
Non-trainable params: 0
_________________________________________________________________


c:\users\admin\appdata\local\programs\python\python38\lib\site-packages\tensorflow\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


Train on 247 samples, validate on 62 samples
Epoch 1/50


KeyboardInterrupt: 

In [ ]:
result={}
resluts={}
print(confusion_matrix(labels[test_indices], predicted_labels))
print(classification_report(labels[test_indices], predicted_labels, digits=4, target_names=namesInLabelOrder))

result['confusion_matrix'] = confusion_matrix(labels[test_indices], predicted_labels).tolist()
result['classification_report'] = classification_report(labels[test_indices], predicted_labels, digits=4, target_names=namesInLabelOrder, output_dict=True)
results[0] = result

os.chdir('D:\\PhD\\Dataset\\Results')
filename = 'LSTM-'+time.strftime("%d-%m-%Y-%H-%M-%S")+'.json'
f = open (filename,'w')
out = json.dumps(results, ensure_ascii=True, indent=4)
f.write(out)
f.close()

In [ ]:
print(len(history.history['acc']))
print(len(history.history['val_acc']))
print(len(history.history['loss']))
print(len(history.history['val_loss']))
print(len(history.history['acc']))

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(8,6))
subplot = fig.add_subplot(1, 1, 1)

loss_train = history.history['loss']
loss_val = history.history['val_loss']

acc_train = history.history['acc']
acc_val = history.history['val_acc']

epochs = range(0,len(history.history['val_acc']))

subplot.plot(epochs,acc_val,color='g', label='Validation')
subplot.plot(epochs,loss_val,color='g')
subplot.plot(epochs,acc_train,color='b')
subplot.plot(epochs,loss_train,color='b', label='Training')

plt.title('Convergence')
plt.xlabel('Epochs')
#subplot.legend(loc='upper left', prop={'size': 9})
plt.legend()
plt.show()

In [ ]:
loss_train = history.history['acc']
loss_val = history.history['val_acc']
epochs = range(0,len(history.history['val_acc']))
plt.plot(epochs, loss_train, 'g', label='Training accuracy')
plt.plot(epochs, loss_val, 'b', label='validation accuracy')
plt.title('Training and Validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
loss_train = history.history['loss']
loss_val = history.history['val_loss']
epochs = range(0,len(history.history['val_loss']))
plt.plot(epochs, loss_train, 'g', label='Training loss')
plt.plot(epochs, loss_val, 'b', label='validation loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()